# Plotting with pandas DataFrames

The pandas DataFrame has the same `plot` method as the Series, and uses the same `kind` parameter to select a plot. All of the Series plots are available to the DataFrame plus scatter and hexbin (a scatter plot that bins data into hexagons). One of the main differences with the DataFrame `plot` method, is that you can choose the columns to use along each axis with the `x` and `y` parameters. 

You can also make plots by not supplying either of these parameters, and if you do, then each column will be plotted independently as the y-values, using the same index for the x-values. Just like most operations in pandas, plotting is **column-based** and you can think of each column as an independent Series. Let's begin by reading in the stocks dataset without putting the date in the index.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('mdap.mplstyle')
stocks = pd.read_csv('../data/stocks/stocks10.csv', parse_dates=['date'])
stocks.head(3)

## Line plots

Most kinds of DataFrame plots allow you to choose between explicitly setting `x` and `y` or implicitly using the index as the x-values and each column as the y-values. Line plots allow both. Here, we plot Microsoft's closing price for each date in the dataset. By default, the column name used for the y-values will be used as the legend label.

In [ ]:
stocks.plot(x='date', y='MSFT', kind='line');

More than one independent line can be plotted by using a list for `y`. A specific qualitative colormap is chosen.

In [ ]:
ax = stocks.plot(x='date', y=['MSFT', 'AAPL', 'WMT'], kind='line', cmap='Dark2')

The returned object is a matplotlib axes which we can use to access each of the lines.

In [ ]:
ax.lines

### Implicitly plotting all columns

Let's put the date in the index and select the two oil companies, Schlumberger and ExxonMobil.

In [ ]:
oil_stocks = stocks.set_index('date')[['SLB', 'XOM']]
oil_stocks.head(3)

We'll call the `plot` method without providing either `x` or `y`. Each column of values is implicitly used as the y-values for an independent line with the index as the x-values for each.

In [ ]:
oil_stocks.plot(kind='line');

## Bar plots

Bar plots work just like line plots. You can explicitly set `x` and `y` or you can implicitly allow pandas to use the index as the x-values and each column as a set of bars of the same color. We'll use the life expectancy data which contains the average life expectancy from nearly every country from 2000 to 2016.

In [ ]:
life_exp = pd.read_csv('../data/life_expectancy.csv')
life_exp.head(3)

Let's find the five countries with the lowest life expectancy for all sexes in the year 2000 and plot every fourth year of data. We use `query` to filter the data and `nsmallest` to select the five lowest countries by life expectancy.

In [ ]:
low_le = life_exp.query('year == 2000 and sex == "all"') \
                 .nsmallest(5, 'life_expectancy')
low_le

We can now run another query on the original data to get life expectancy every fourth year.

In [ ]:
low_countries = low_le['country']
low_le_4th = life_exp.query("country in @low_countries and year % 4 == 0 and sex == 'all'")
low_le_4th.head(10)

Getting the plot we desire isn't possible with the current structure of the data. Using the countries as the x-values allows us to infer the year, but still doesn't make a good plot.

In [ ]:
low_le_4th.plot(x='country', y='life_expectancy', kind='bar');

We need to make each column a country with values equal to the life expectancy. Let's use the `pivot` method to change the structure of the data.

In [ ]:
country_le = low_le_4th.pivot(index='year', columns='country', values='life_expectancy')
country_le

We can now use pandas implicit plotting so that every column is plotted as its own set of unique bars.

In [ ]:
ax = country_le.plot(kind='bar', title='Life Expectancy', cmap='Paired')

Each country now has its own set of bars, but the legend is taking up too much space. pandas internally supplies the `label` to each set of bars and calls matplotlib's axes `legend` method for us. We can verify that the label has been set by retrieving the list of containers (there are five total, one for each country), selecting one of the elements and using a getter method to return the label.

In [ ]:
ax.containers[0].get_label()

Since the labels are already set, we can just call the `legend` method again from the axes and use `bbox_to_anchor` to specify an exact location for the legend.

In [ ]:
ax.legend(bbox_to_anchor=(1, .9), loc='upper left', title='Country')
ax.figure

A line plot is a better choice to see the trend by country. Adding markers is important to denote the exact position of the data we are plotting. Each tick, which was originally placed two years apart, is changed to every four years.

In [ ]:
from matplotlib import ticker
ax = country_le.plot(kind='line', title='Life Expectancy', cmap='Paired', marker='.')
ax.legend(bbox_to_anchor=(1, .9), title='Country')
ax.xaxis.set_major_locator(ticker.MultipleLocator(4))

## Plotting on separate axes

Instead of plotting DataFrame columns on the same axes, each one can be plotted on a separate axes by setting the `subplots` parameter to `True`. Set the `layout` parameter to a two-item integer tuple of the number of rows and columns. The boolean parameters `sharex` and `sharey`, when set to `True`, ensure that the limits and ticks of the respective axis are the same. 

Each country is plotted on its own axes. By sharing both the x and y axis, comparing the countries becomes much easier. Also, only the left axes have y-tick labels It appears as though only five axes were created, one for each country, but any axes that doesn't contain data has the attribute `visible` set to `False`. pandas returns a numpy array with each of the six axes we created. The original x-axis tick marks were set at five year intervals. Since all of the x-axises are shared, changing one changes them all.

In [ ]:
ax_array = country_le.plot(kind='line', cmap='Paired', title='Life Expectancy', 
                           marker='.', figsize=(6, 3), layout=(3, 2), subplots=True, 
                           legend=False, sharex=True, sharey=True)
ax1 = ax_array[0, 0]
fig = ax1.figure
fig.legend(bbox_to_anchor=(.92, .8), loc='upper left', bbox_transform=fig.transFigure)
ax1.xaxis.set_major_locator(ticker.MultipleLocator(4))
ax1.minorticks_off()

## Distribution plots

The same three distribution plots, box, histogram, and KDE are available for DataFrames. They work a little differently as the other plots as they disregard the `x` parameter and only use `y`. Let's read in the neighborhood and sale price along with several columns containing information on square footage from the housing dataset.

In [ ]:
cols = ['Neighborhood', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 
        'GrLivArea', 'GarageArea' , 'LotArea', 'SalePrice']
housing = pd.read_csv('../data/housing.csv', usecols=cols)
housing.head(3)

### Box plots

Without specifying `x` or `y`, a box plot is created for each numeric column. All non-numeric columns, such as neighborhood, are silently dropped. The x-limits (or y-limits when vertical) will range from the overall lowest to highest values from all variables, so a single variable like `LotArea` or `SalePrice` which are around 1,000 times greater than the others can dominate the plot.

In [ ]:
housing.plot(kind='box', vert=False);

You can choose specific columns to plot with the `y` (and NOT `x`) parameter by setting it to a list. We also extend the whiskers to three times the interquartile range.

In [ ]:
housing.plot(y=cols[1:-2], kind='box', vert=False, whis=3);

### Histograms

Each of the five columns in the last box plot is made into a cumulative step histogram with the relative frequency on the y-axis. From the graph you can see that close to 60% of houses have no second floor and that nearly all houses have less than 1,000 total basement square feet.

In [ ]:
ax = housing.plot(y=cols[1:-2], kind='hist', bins=50, range=(0, 3_000), alpha=.6, 
                  histtype='step', cumulative=True, lw=1.5, density=True)
ax.set_xlabel('Square Feet')
ax.legend(bbox_to_anchor=(-.1, 1.2), loc='upper left', ncol=5,
          handlelength=1, columnspacing=.8);

### KDEs

KDEs work similarly as histograms. Each column is plotted as an independent KDE.

In [ ]:
ax = housing.plot(y=cols[1:-2], kind='kde', alpha=.6, lw=1.5, xlim=(0, 3_000), legend=True)
ax.set_xlabel('Square Feet');

## Scatter and Hexbin

Scatter and hexbin plots are only possible with DataFrames, not with Series. They both work differently than the plots above and **require** you to set `x` and `y` to a single column name. Below, we make a scatter plot of garage area versus living area using the `c` parameter to color by sale price. A colorbar is also created automatically. There's currently a bug in pandas requiring us to manually make the x-axis label visible and to set the tick position.

In [ ]:
ax = housing.plot(x='GarageArea', y='GrLivArea', kind='scatter', s=10, 
                  c='SalePrice', cmap='RdYlGn_r', alpha=.5)
ax.xaxis.label.set_visible(True)
ax.tick_params(axis='x', labelbottom=True)

Hexbin plots are similar to scatter plots. They divide up the region they are plotting into equally-sized hexagons, and count the number of observations that fall within the bounds of each hexagon. These hexagons are then filled in with a color corresponding to the number of observations in that bin. pandas uses the [matplotlib `hexbin` plot, so check out its documentation to learn about all of its parameters][0].

The `gridsize` parameter controls the number of hexagons to create along the x and y directions. If a single integer is provided, it is used as the number in the x direction with matplotlib computing the number in the y-direction so that the hexagons are regular. Set it to a tuple to control the exact number in both directions.

By default, pandas uses the `BuGn` sequential colormap. Below, we choose a different colormap, the reverse of `gist_heat`. The `mincnt` parameter controls the minimum count needed for a hexagon to be shown. This is useful when you have a colormap that does not have white as its minimum value. It prevents filling hexagons with a color if they have no count.

[0]: https://matplotlib.org/api/_as_gen/matplotlib.pyplot.hexbin.html

In [ ]:
ax = housing.plot(x='GarageArea', y='GrLivArea', kind='hexbin', 
                  xlim=(-100, 1_200), ylim=(0, 4_000), title='Hexbin House Count',
                  cmap='gist_heat_r', gridsize=25, mincnt=1)
ax.xaxis.label.set_visible(True)
ax.tick_params(axis='x', labelbottom=True)

pandas automatically adds a colorbar to the figure for us. The darkest hexagons have 40 or more houses with those particular combinations for garage area and living area. Hexbins are quite useful alternatives for scatter plots whenever there are many overlapping points. They can help get a better feel for the actual distribution.

The above plot merely counted the observations in each hexagon. It's possible to aggregate another variable and use the result for the color instead of the count. Set the `C` parameter to the column name you want to aggregate. By default, the average value of this variable will be calculated. Below, the average sale price is used to color the hexagons. The colorbar axes is selected and labels are formatted.

In [ ]:
ax = housing.plot(x='GarageArea', y='GrLivArea', C='SalePrice', kind='hexbin', 
                  xlim=(-100, 1_200), ylim=(0, 4_000), title='Hexbin Average Sale Price',
                  cmap='gist_heat_r', gridsize=25, mincnt=5)
ax.xaxis.label.set_visible(True)
ax.tick_params(axis='x', labelbottom=True)
ax_colorbar = ax.figure.axes[-1]
ax_colorbar.yaxis.set_major_formatter(ticker.StrMethodFormatter('${x!s:.3}k'))

### Relative representation of each sex by experience and salary

In this section, we'll divide City of Houston employees into bins by experience and salary and report the relative representation of each sex. For instance, 70% of employees are male. If the group of employees with 10 years of experience and a salary of 80,000 is 60% male, then males are underrepresented by 10%. Let's read in the data and create a column for years of experience.

In [ ]:
emp = pd.read_csv('../data/employee.csv', parse_dates=['hire_date'])
emp['experience'] = 2019 - emp['hire_date'].dt.year
emp.head(3)

Let's get the relative frequency of each sex and verify that 70% of employees are male.

In [ ]:
emp['sex'].value_counts(normalize=True).round(2)

Before finding the relative representation of each sex, we will find the percent male per group using a hexbin plot. We create a numeric binary column equal to 1 for male employees and pass this to the `C` parameter, which is aggregated with the mean function by default. The divergent colormap 'Spectral' is chosen since there are two unique values for sex. To ensure that the midpoint of the colormap is .5, `vmin` and `vmax` are set to 0 and 1. A minimum of 5 employees is set for the hexagon to be shown.

In [ ]:
emp['is_male'] = (emp['sex'] == 'Male') * 1
ax = emp.plot(x='experience', y='salary', C='is_male', kind='hexbin', 
              title='Percent Male', xlim=(-1, 40), ylim=(0, 200_000),
              gridsize=20, cmap='Spectral', vmin=0, vmax=1, mincnt=5)
ax.xaxis.label.set_visible(True)
ax.tick_params(axis='x', labelbottom=True)
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: f'${x / 1000:.0f}k'))

To get the relative representation of each group, we'll need to write our own custom aggregation function and pass it to the `reduce_C_function` parameter. matplotlib implicitly passes this function a list of all the values of the `C` column for that group. 

In our custom function `relative_male`, we calculate the relative frequency of males and subtract the overall relative frequency of males calculated above (.7). This gives us a metric of over/under representation of each sex per group. Values greater than 0 have an over-representation of males (more than 70%) while those less than 0 are under-represented.

In [ ]:
def relative_male(x):
    return pd.Series(x).mean() - .7

ax = emp.plot(x='experience', y='salary', C='is_male', kind='hexbin', 
              title='Relative Representation by Sex', xlim=(-1, 40), ylim=(0, 200_000),
              gridsize=20, cmap='Spectral', vmin=-.3, vmax=.3, mincnt=5,
              reduce_C_function=relative_male)
ax.xaxis.label.set_visible(True)
ax.tick_params(axis='x', labelbottom=True)
ax.yaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: f'${x / 1000:.0f}k'))

## Area plots

Area plots are a great way to show emerging trends over time. In this section, we'll make different area plots based on the currently ongoing COVID-19 pandemic. Let's read in data collected by the World Health Organization (WHO) with new deaths by day for each country.

In [ ]:
c19_new_deaths = pd.read_csv('../data/covid/new_deaths.csv', index_col='date', 
                             parse_dates=['date'])
c19_new_deaths.tail(3)

To get the total deaths for each date, take the cumulative sum of each column.

In [ ]:
c19_total = c19_new_deaths.cumsum()
c19_total.tail(3)

Making a plot containing all of the more than 180 countries would be very messy. Let's select the five countries with most deaths at the current time by sorting horizontally by the last index date.

In [ ]:
last_date = c19_total.index[-1]
c19_total_top = c19_total.sort_values(last_date, axis=1, ascending=False)
c19_total_top = c19_total_top.iloc[:, :6]
c19_total_top.tail(3)

Let's create a column called 'Rest of World' and set it equal to the total of all deaths outside of these countries and then drop the 'World' column.

In [ ]:
c19_total_top['Rest of World'] = c19_total_top['World'] - c19_total_top.iloc[:, 1:].sum(axis=1)
c19_total_top = c19_total_top.drop(columns='World')
c19_total_top.tail(3)

We can now create an area plot that draws a line for each column beginning with the first column. Each subsequent column uses the cumulative total of all y-values before it. The area between each lines is filled in with a different color. This vertical area represents the number of deaths for each country at each date. We begin the plot in mid-February after a larger number of deaths had already occurred.

In [ ]:
c19_total_top_recent = c19_total_top['2020-02-17':]
c19_total_top_recent.plot(kind='area', title='Total COVID-19 Deaths', cmap='Paired');

Instead of plotting the total number of deaths, we can plot each country's percentage of the total world deaths per day. To get these percentages, we first sum horizontally to get the total deaths in the world and then divide each row by its corresponding world total. We must use the `div` method to change the direction of the division so that the rows align.

In [ ]:
total_world = c19_total_top_recent.sum(axis=1)
c19_perc_top_recent = c19_total_top_recent.div(total_world, axis=0)
c19_perc_top_recent.tail(3).round(2)

Another area plot can be made. The height of the stacked lines must sum to 1 each day. Using a relative metric allows us to use more space on the plotting surface and show emerging trends clearer.

In [ ]:
ax = c19_perc_top_recent.plot(kind='area', title='Percentage of Total COVID-19 Deaths', 
                              cmap='Paired', figsize=(6, 3), legend=True, ylim=(0, 1))
ax.set_xlabel('')
ax.grid(True, axis='y')

Instead of having a legend, we can place text on the right side of the axes denoting the country and the last percentage. Let's get the last row of data.

In [ ]:
last_perc = c19_perc_top_recent.iloc[-1]
last_perc.round(3)

We remove the legend and iterate through the country name and associated value by converting the Series to a dictionary. We use axes coordinates (`ax.transAxes`) to place the text just outside of the axes as the y-values are in the same units for both the data and axes coordinates, making it possible to use the percentages to make the placement.

In [ ]:
ax.legend_.remove()
y = 0
for country, perc in last_perc.to_dict().items():
    ax.text(x=1.01, y=y + perc / 2, s=f'{country} - {perc:.1%}', 
            ha='left', va='center', transform=ax.transAxes)
    y += perc
ax.figure

matplotlib allows you to add tables of data directly below your plots with the `table` axes method. We create our table by selecting five rows spaced out at even intervals and then transpose it so the dates are horizontal.

In [ ]:
step = len(c19_total_top_recent) // 5 + 1
table = c19_total_top_recent[::step].T.astype('int64')
table

The axes `table` method is now called needing the cell data passed as a numpy array. Other parameters are used to set the row and column labels. The `bbox` parameter sets the rectangular region (left, bottom, width, height) where the table will be placed.

In [ ]:
ax.table(cellText=table.values, rowLabels=table.index, 
         colLabels=table.columns.astype('str'), bbox=(0, -.8, 1, .5))
ax.figure

We can collect different insights by plotting the percentage of new deaths by country instead of the total. Let's begin by going back to our original data and finding the top 9 countries by total number of deaths.

In [ ]:
top10 = c19_new_deaths.sum().nlargest(10)
top10

We select these countries and then calculate a three day rolling average of the new deaths. We do this because daily data is erratic and subject to errors. Smoothing it out using three days of data will prevent jagged spikes in our plot.

In [ ]:
new_deaths_top = c19_new_deaths[top10.index].copy()
new_deaths_top_avg = new_deaths_top.rolling(3, center=True, min_periods=1).mean()
new_deaths_top_avg.tail(3).round(0)

We calculate the 'Rest of World' column again.

In [ ]:
new_deaths_top_avg['World'] = new_deaths_top_avg['World'] - new_deaths_top_avg.iloc[:, 1:].sum(1)
new_deaths_top_avg = new_deaths_top_avg.rename(columns={'World': 'Rest of World'})
new_deaths_top_avg.tail(3).round(0)

Dividing by the daily total yields the proportion per day.

In [ ]:
new_deaths_top_perc = new_deaths_top_avg.div(new_deaths_top_avg.sum(axis=1), axis=0)
new_deaths_top_perc.tail(3).round(2)

We make another area plot to reveal a different emerging trend. China's contribution to the daily total has shrunk from all to nearly none.

In [ ]:
pct_daily_final = new_deaths_top_perc['2020-02-17':]
ax = pct_daily_final.plot(kind='area', figsize=(6, 3), ylim=(0, 1), legend=False,
                          cmap='Paired', title='COVID-19 - Proportion of New Deaths')
ax.grid(axis='y')
ax.legend(bbox_to_anchor=(1, 1), loc='upper left')
ax.set_xlabel('');